In [1]:
import pandas as pd
import numpy as np

## Metadata

In [2]:
PATH_TRAIN = "source/evo_train.csv"

In [3]:
target_label = u'GROUP_ID'
pred_labels = u'NAME'

## Preparing data ##

In [4]:
dtrain = pd.read_csv(PATH_TRAIN)

In [5]:
target = dtrain[target_label]
preds = dtrain[pred_labels]

### Partition

In [24]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    preds, target, 
    test_size=0.33, 
    random_state=42)

### Functions

In [7]:
def check_accuracy(clf, x, y, x_test, y_test):
    clf.fit(x, y)
    t = clf.predict(x_test)
    from sklearn.metrics import accuracy_score
    print "accuracy_score: %f" % accuracy_score(y_test, t)

## Model

### Baseline ###

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
baseline_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB())])
check_accuracy(baseline_clf, X_train, y_train, X_test, y_test)

accuracy_score: 0.812249


### Best model

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
clf_v4 = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer(use_idf=False)),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))])
check_accuracy(clf_v4, X_train, y_train, X_test, y_test)

accuracy_score: 0.892210


## Best features

In [73]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline


clf_v5 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer(use_idf=False)),
                   ('ch2', SelectKBest(chi2, k=5000)),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))
                  ])

# t = clf_v5.fit_transform(X_train, y_train)
check_accuracy(clf_v5, X_train, y_train, X_test, y_test)

accuracy_score: 0.872898


In [99]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline


clf_v5 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer(use_idf=False)),
                   ('select', SelectFromModel(LogisticRegression(C=0.01, penalty="l2", dual=False), threshold="0.01*mean")),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))
                  ])

# t = clf_v5.fit_transform(X_train, y_train)
check_accuracy(clf_v5, X_train, y_train, X_test, y_test)

accuracy_score: 0.907779


In [54]:
counter = CountVectorizer()
t = counter.fit_transform(X_train)
k=0.01 * t.shape[1]
k=100
chi = SelectKBest(chi2, k=k)
t =chi.fit_transform(t, y_train)
print t.shape
print t.shape[1]

(37426, 100)
100


In [79]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

counter = CountVectorizer()
t = counter.fit_transform(X_train)

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(t, y_train)
model = SelectFromModel(lsvc)
X_new = model.fit_transform(t)
X_new.shape

ValueError: bad input shape ()

In [76]:
pd.Series(counter.vocabulary_)[model.get_support()]

0mg             380
10              396
100г            453
100мл           461
150мл          1391
1кг            1851
1л             1857
20             1896
30             2799
30ml           2876
30мл           2888
37             3241
3mg            3333
3мг            3368
40             3389
5л             4345
75             4874
adalya         5835
blue           6549
compact        7181
eleaf          7821
fanatik        8049
giay           8480
gsh            8641
iphone         9062
mg            10092
red           11324
silver        11862
smok          11945
strobbs       12204
              ...  
тени          33511
термос        33560
тетрадь       33612
товар         33733
томат         33768
тон           33785
трости        34076
тумба         34149
турка         34166
туф           34182
туфли         34185
тушь          34196
унитаза       34475
услуги        34588
филе          34932
фонарь        35142
хлеб          35553
цифра         35946
чай           35997


In [55]:
print len(chi.get_support())
print len(counter.vocabulary_)

37474
37474


In [56]:
pd.Series(counter.vocabulary_)[chi.get_support()]

00               0
5л            4345
6s            4707
bb            6345
bg            6440
fanatik       8049
ip            9050
iphone        9062
vandoren     12741
алк          13706
альта        13794
анальная     13892
белое        14893
ботинки      15442
брелок       15508
вешалка      16298
винный       16413
вино         16414
водка        16645
волос        16752
вох          16841
гавриш       17081
гер          17271
горшок       17678
детские      18412
для          18653
жен          19297
жидкость     19368
заказ        19547
защитное     19724
             ...  
саксофона    30835
сапоги       30983
сверло       31100
светлое      31117
свободная    31204
свободной    31205
свободные    31206
семена       31363
серьги       31504
сигареты     31565
собак        32113
сопрано      32266
столовое     32780
сувенир      32942
сумка        32978
таб          33236
табл         33243
тенора       33519
тетрадь      33612
трости       34076
тумба        34149
туфли       

## Batch

In [10]:
from sklearn.utils import resample

n_samples = 10000

Bpreds, Btarget = resample(preds, target, 
                           replace=False,
                           n_samples=n_samples,
                           random_state=17)

In [11]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    Bpreds, Btarget, 
    test_size=0.33, 
    random_state=42)

### Baseline ###

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
baseline_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB())])
check_accuracy(baseline_clf, X_train, y_train, X_test, y_test)

accuracy_score: 0.721515


### Best model

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
clf_v4 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer(use_idf=False)),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))])
check_accuracy(clf_v4, X_train, y_train, X_test, y_test)

accuracy_score: 0.853636


In [14]:
import re
def replace_digits(line):
    pat = ur'[\d]{2,1000}([\W][\d]+)*'
    repl = " "
    return re.sub(pat, repl, line)

In [15]:
repl = [" ", " %DIGIT% "]
max_df = [0.5, None]
use_idf = [True, False]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
clf_v4 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer()),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))])
check_accuracy(clf_v4, X_train.apply(lambda x: replace_digits(x)), y_train, X_test.apply(lambda x: replace_digits(x)), y_test)

accuracy_score: 0.857879


In [18]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def normal_form(word):
    return morph.parse(word)[0].normal_form

def normal_form_line(line):    
    return u' '.join([normal_form(unicode(x, 'utf-8')) for x in line.split()])

ImportError: No module named pymorphy2

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
clf_v4 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer()),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))])
check_accuracy(clf_v4, X_train.apply(lambda x: replace_digits(x)).apply(lambda x: normal_form_line(x)), y_train, X_test.apply(lambda x: replace_digits(x)).apply(lambda x: normal_form_line(x)), y_test)

NameError: global name 'normal_form_line' is not defined

In [ ]:
vect =  CountVectorizer()
t = vect.fit_transform(X_train.apply(lambda x: replace_digits(x)))
vocab = pd.Series(vect.vocabulary_)

In [ ]:
vocab.count()

In [ ]:
pd.DataFrame(t.todense())

In [ ]:
vocab.to_csv("vocab_batch.csv", encoding="UTF-8", sep='\t')